In [1]:
# Add parent directory to sys.path so we can import wikipedia_downloader.py
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
sys.path.append(os.path.abspath(os.getcwd()))

In [2]:
# Import the Wikipedia downloader module
from wikipedia_downloader import download_wikipedia_to_markdown, get_page_title_from_url

# Example usage: Download a Wikipedia page
wikipedia_url = "https://en.wikipedia.org/wiki/Artificial_intelligence"
output_file = "downloads/artificial_intelligence.md"

# Download the page
success = download_wikipedia_to_markdown(wikipedia_url, output_file)

if success:
    print(f"Wikipedia page downloaded successfully!")
    
    # Read and display first 500 characters of the downloaded content
    with open(output_file, 'r', encoding='utf-8') as f:
        content = f.read()
        print(f"\nFirst 500 characters of the downloaded content:")
        print(content[:500] + "..." if len(content) > 500 else content)
else:
    print("Failed to download Wikipedia page")

Wikipedia page downloaded successfully!

First 500 characters of the downloaded content:
Intelligence of machines
"AI" redirects here. For other uses, see AI (disambiguation) and Artificial intelligence (disambiguation).
**Artificial intelligence** (**AI**) is the capability of computational systems to perform tasks typically associated with human intelligence, such as learning, reasoning, problem-solving, perception, and decision-making. It is a field of research in computer science that develops and studies methods and software that enable machines to perceive their environment an...
